In [344]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *
from sklearn.metrics import *
from scipy.stats import linregress

from sklearn.metrics.pairwise import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [347]:
%%time
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])
df = dataPrep(ings, prods, prod_ing)

print('Number of products df: ', len(df))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products df:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618
Wall time: 3.5 s


In [364]:
%%time
##Long dataframes
df2_A = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
df2_B = pd.read_csv('data/prodIng_Wide_02.csv', index_col=0)
df2_C = pd.read_csv('data/prodIng_Wide_03.csv', index_col=0)
frames = [df2_A, df2_B, df2_C]
df2 = pd.concat(frames)
df2 = df2.T

Wall time: 21.8 s


# GET RECOMMENDATIONS


In [307]:
product ='C E Ferulic Serum'

##get indices of all products
indices = pd.Series(df.index, index=df['product']).drop_duplicates()
##find index of specified product
idx = indices[product]

print('Product:', df.iloc[idx]['product'])
print('Brand:', df.iloc[idx]['brand'])
print('Id:', idx)
print('Number of ingredients:', df.iloc[idx]['ingCount'])
print('Ingredients', df.iloc[idx]['ingList'])

Product: C E Ferulic Serum
Brand: SkinCeuticals
Id: 4350
Number of ingredients: 12
Ingredients ['Water', 'Ethoxydiglycol', 'Ascorbic Acid (Vitamin C', 'Propylene Glycol', 'Glycerin', 'Laureth-23', 'Tocopherol (Vitamin E', 'Phenoxyethanol', 'Triethanolamine', 'Ferulic Acid', 'Panthenol', 'Sodium Hyaluronate']


#### Find product id with name

In [316]:
product_find = '20% Vitamin C + E Ferulic Acid Serum'
idx_2 = indices[product_find]
#idReturn = row.index.item()

print('Product:', df.iloc[idx_2]['product'])
print('Brand:', df.iloc[idx_2]['brand'])
print('Id:', idx_2)
print('Number of ingredients:', df.iloc[idx_2]['ingCount'])
print('Ingredients', df.iloc[idx_2]['ingList'])

Product: 20% Vitamin C + E Ferulic Acid Serum
Brand: Timeless
Id: 449
Number of ingredients: 11
Ingredients ['Water', 'Ethoxydiglycol', 'Ascorbic Acid (Vitamin C', 'Propylene Glycol', 'Tocopherol (Vitamin E', 'Polysorbate 80', 'Panthenol', 'Ferulic Acid', 'Sodium Hyaluronate', 'Benzyl Alcohol', 'Dehydroacetic Acid']


In [312]:
idx_2

4350

## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

### Create dataframe of top ten similar products

In [327]:
%%time
column = 'product'
# Get the index of the product that matches the product name
df['ing#List'] = df['ing#List'].astype(str)
tfidf = TfidfVectorizer(binary = True, stop_words = None)
tfidf_matrix = tfidf.fit_transform(df['ing#List'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df[column]).drop_duplicates()
idx = indices[product]

# Get the pairwsie similarity scores of all products with that product
sim_scores_all = list(enumerate(cosine_sim[idx]))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

# Return the top 10 most similar products and their ingredients
#return df[['product', 'brand','ingList']].iloc[product_indices]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 29.9 s


### Show top ten

In [328]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/COS.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,13443,Ultimate Antioxidant C Boost Serum,Youth Corridor,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",11
1,449,20% Vitamin C + E Ferulic Acid Serum,Timeless,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
2,3918,Serum 20 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
3,3995,Serum 10 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
4,4104,Serum 15 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
5,11015,Vitamin C Infusion,Pulsaderm,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",10
6,15419,RESIST C15 Super Booster,Paula's Choice,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",10
7,8123,"Vitamin C, E, Ferulic And Hyaluronic Acid Serum",Bombshell Bathtime,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",8
8,8482,Brightening Elixir,Goldfaden MD,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
9,8097,C Perfect Vitamin C Serum,Singularity Skin,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8


### Find specific product similarity

In [335]:
prod_search_id = 3918
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('Cosine Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
print('Number of Common Ingredients', len(set(list1)&set(list2)))
print('Ingredients', df.iloc[prod_search_id]['ingList'])

Cosine Similarity
---------------------------------------------------------------
Product: Serum 20 AOX+
Similarity score: 0.674559612989
Rank: 3
Number of Ingredients 11
Number of Common Ingredients 9
Ingredients ['Water', 'Ascorbic Acid (Vitamin C', 'Ethoxydiglycol', 'Propylene Glycol', 'Glycerin', 'Zinc Sulfate', 'Phenoxyethanol', 'Ferulic Acid', 'Panthenol', 'Sodium Hyaluronate', 'Sodium Hydroxide']


## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

### Create dataframe of top ten similar products

In [348]:
%%time

##Set the rbo we want to use
rbo = rbo_min

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [rbo(itemLookup,i, .9) for i in items] ##change method here

sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 2.4 s


### Show top ten

In [349]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,13443,Ultimate Antioxidant C Boost Serum,Youth Corridor,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",11
1,11015,Vitamin C Infusion,Pulsaderm,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",10
2,3918,Serum 20 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
3,3995,Serum 10 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
4,4104,Serum 15 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
5,449,20% Vitamin C + E Ferulic Acid Serum,Timeless,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
6,8097,C Perfect Vitamin C Serum,Singularity Skin,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
7,8482,Brightening Elixir,Goldfaden MD,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
8,2218,C-Firma Day Serum,Drunk Elephant,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",9
9,7784,C20 + Ferulic Vitamin C Serum,NuFountain,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",7


### Find specific product similarity

In [356]:
prod_search_id = 3918
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('RBO Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
#print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
#print('Number of Common Ingredients', len(set(list1)&set(list2)))
#print('Ingredients', df.iloc[prod_search_id]['ingList'])

RBO Similarity
---------------------------------------------------------------
Product: Serum 20 AOX+
Similarity score: 0.7067781966364534
Rank: 4


## R value
Reference [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.linregress.html)

In [362]:
df2[idx]

,4350,4350,4350
"ing_ ""Coix Lacryma-Jobi Ma-yuen (Job's Tears) Seed Oil""",0,0,0
"ing_ ""Fuller's Earth""",0,0,0
ing_ ',0,0,0
ing_ '-)-alpha-isabolol',0,0,0
ing_ '1,0,0,0
ing_ '1-Methylhydantoin-2-Imide',0,0,0
ing_ '10-Hydroxydecanoic Acid',0,0,0
ing_ '2,0,0,0
ing_ '2-Bromo-2-Nitropropane-1,0,0,0
ing_ '2-O-Ethyl Ascorbic Acid',0,0,0


In [365]:
%%time
#get linregress similarity and return r-value
sim = [linregress(df2[idx], df2[i])[2] for i in df2]

##collect all similarity scores (r_val)
sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)
#df['scores'] = [sim_scores[i][1] for i in sim_scores]

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 38.2 s


### Return top ten

In [366]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,13443,Ultimate Antioxidant C Boost Serum,Youth Corridor,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",11
1,3918,Serum 20 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
2,3995,Serum 10 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
3,4104,Serum 15 AOX+,SkinCeuticals,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",9
4,449,20% Vitamin C + E Ferulic Acid Serum,Timeless,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
5,15419,RESIST C15 Super Booster,Paula's Choice,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",10
6,8482,Brightening Elixir,Goldfaden MD,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
7,11015,Vitamin C Infusion,Pulsaderm,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",10
8,8097,C Perfect Vitamin C Serum,Singularity Skin,"[Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...",8
9,8123,"Vitamin C, E, Ferulic And Hyaluronic Acid Serum",Bombshell Bathtime,"[Water, Ascorbic Acid (Vitamin C, Ethoxydiglyc...",8


### Find specific product similarity

In [374]:
prod_search_id = 3918
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('RVal Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
#print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
#print('Number of Common Ingredients', len(set(list1)&set(list2)))
#print('Ingredients', df.iloc[prod_search_id]['ingList'])

RVal Similarity
---------------------------------------------------------------
Product: Serum 20 AOX+
Similarity score: 0.783027568469
Rank: 2
